<a href="https://colab.research.google.com/github/JunyeChen2025/Camera_Privacy_Shield/blob/main/Face_Detection_Mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q '/content/drive/My Drive/YOLO_Data/Wider_Face_Dataset.zip' -d /content/Dataset

In [3]:
import os
import shutil

# Set Path
base_path = "/content/Dataset/Wider_Face_Dataset"
output_path = os.path.join(base_path, "images")
os.makedirs(output_path, exist_ok=True)

# Define the mapping of source directories to target subfolders
folder_map = {
    "WIDER_train": "train",
    "WIDER_val": "val",
    "WIDER_test": "test"
}

for src_folder, target_subfolder in folder_map.items():
    src_path = os.path.join(base_path, src_folder)
    dst_path = os.path.join(output_path, target_subfolder)
    os.makedirs(dst_path, exist_ok=True)

    # Iterate over subdirectories
    for root, dirs, files in os.walk(src_path):
        for file in files:
            if file.lower().endswith(".jpg"):
                src_file_path = os.path.join(root, file)
                dst_file_path = os.path.join(dst_path, file)
                shutil.copy2(src_file_path, dst_file_path)

    print(f"Extraction completed：{src_folder} -> images/{target_subfolder}")

# Delete original folders
for folder in folder_map.keys():
    folder_path = os.path.join(base_path, folder)
    shutil.rmtree(folder_path)
    print(f"Deleted orginal directory：{folder_path}")

print("All images have been collated.")


Extraction completed：WIDER_train -> images/train
Extraction completed：WIDER_val -> images/val
Extraction completed：WIDER_test -> images/test
Deleted orginal directory：/content/Dataset/Wider_Face_Dataset/WIDER_train
Deleted orginal directory：/content/Dataset/Wider_Face_Dataset/WIDER_val
Deleted orginal directory：/content/Dataset/Wider_Face_Dataset/WIDER_test
All images have been collated.


In [4]:
import os
from PIL import Image

def convert_widerface_to_yolo(txt_path, image_base_dir, save_label_dir, class_id=0):
    os.makedirs(save_label_dir, exist_ok=True)

    with open(txt_path, 'r') as f:
        lines = f.readlines()

    idx = 0
    while idx < len(lines):
        filename = lines[idx].strip()
        idx += 1

        if idx >= len(lines):
            break

        try:
            face_count = int(lines[idx].strip())
        except ValueError:
            print(f"Skipping invalid face count at line {idx}: {lines[idx]}")
            idx += 1
            continue
        idx += 1

        label_lines = []

        image_path = os.path.join(image_base_dir, os.path.basename(filename))
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}, skipping.")
            idx += face_count
            continue

        try:
            img = Image.open(image_path)
            w_img, h_img = img.size
        except:
            print(f"Failed to open image: {image_path}")
            idx += face_count
            continue

        for _ in range(face_count):
            if idx >= len(lines):
                break
            parts = lines[idx].strip().split()
            if len(parts) < 4:
                idx += 1
                continue

            try:
                x, y, w, h = map(float, parts[:4])
                x_center = (x + w / 2) / w_img
                y_center = (y + h / 2) / h_img
                w_norm = w / w_img
                h_norm = h / h_img
                label_line = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
                label_lines.append(label_line)
            except:
                pass
            idx += 1

        # Save label
        base_name = os.path.splitext(os.path.basename(filename))[0]
        label_file = os.path.join(save_label_dir, base_name + '.txt')
        with open(label_file, 'w') as f:
            f.write('\n'.join(label_lines))


In [5]:
# Set Path
base_path = "/content/Dataset/Wider_Face_Dataset"
anno_path = os.path.join(base_path, "Annotations")
image_dir = os.path.join(base_path, "images")

# Convert training set and validation set
convert_widerface_to_yolo(
    txt_path=os.path.join(anno_path, "wider_face_train_bbx_gt.txt"),
    image_base_dir=os.path.join(image_dir, "train"),
    save_label_dir=os.path.join(base_path, "labels/train")
)

convert_widerface_to_yolo(
    txt_path=os.path.join(anno_path, "wider_face_val_bbx_gt.txt"),
    image_base_dir=os.path.join(image_dir, "val"),
    save_label_dir=os.path.join(base_path, "labels/val")
)


Skipping invalid face count at line 10424: 0--Parade/0_Parade_Parade_0_630.jpg

Skipping invalid face count at line 10426: 20 186 14 20 1 0 0 0 0 0 

Skipping invalid face count at line 10428: 119 181 14 17 2 0 0 0 2 0 

Skipping invalid face count at line 10430: 375 129 46 62 0 0 0 0 0 0 

Skipping invalid face count at line 10432: 349 122 26 38 0 0 0 0 0 0 

Skipping invalid face count at line 10434: 527 174 10 16 1 0 0 0 0 0 

Skipping invalid face count at line 10436: 695 143 49 64 0 0 0 0 0 0 

Skipping invalid face count at line 10438: 822 138 29 38 0 0 0 0 2 0 

Skipping invalid face count at line 10440: 954 156 12 18 1 0 0 0 0 0 

Skipping invalid face count at line 10442: 615 143 22 24 1 0 0 0 2 0 

Skipping invalid face count at line 86539: 2--Demonstration/2_Demonstration_Political_Rally_2_414.jpg

Skipping invalid face count at line 86541: 52 130 120 204 0 0 0 0 0 1 

Skipping invalid face count at line 86543: 400 156 62 64 1 0 0 0 1 1 

Skipping invalid face count at line 

In [6]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model.train(
    data="/content/Dataset/Wider_Face_Dataset/wider_face.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    name="face_yolov8"
)

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Dataset/Wider_Face_Dataset/wider_face.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=face_yolov83, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

100%|██████████| 755k/755k [00:00<00:00, 24.4MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 80.0MB/s]


...

val: /content/Dataset/Wider_Face_Dataset/images/val/21_Festival_Festival_21_604.jpg: 1 duplicate labels removed
val: New cache created: /content/Dataset/Wider_Face_Dataset/labels/val.cache


Plotting labels to runs/detect/face_yolov83/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/face_yolov83
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30      11.9G      2.981      2.852      2.384        238        640: 100%|██████████| 805/805 [04:42<00:00,  2.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:37<00:00,  2.73it/s]


                   all       3226      39707      0.424      0.229      0.199     0.0782

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/30        13G      2.164      1.555      1.431        171        640:  97%|█████████▋| 778/805 [04:16<00:11,  2.27it/s]